In [1]:
import requests
from PIL import Image
import json
import sqlite3
import pandas as pd
from io import BytesIO



In [2]:
df = pd.read_json("./breed_list.json")

In [3]:

df['imperial_weight'] = df['weight'].apply(lambda x: x['imperial'])
df['metric_weight'] = df['weight'].apply(lambda x: x['metric'])
df['imperial_height'] = df['height'].apply(lambda x: x['imperial'])
df['metric_height'] = df['height'].apply(lambda x: x['metric'])

In [4]:
df['image_id'] = df['image'].apply(lambda x: x['id'])
df['image_url'] = df['image'].apply(lambda x: x['url'])
df['image_width'] = df['image'].apply(lambda x: x['width'])
df['image_height'] = df['image'].apply(lambda x: x['height'])

In [5]:
df.drop(columns=['weight', 'height', 'origin', 'country_code', 'description', 'history', 'image' ], inplace=True)

In [ ]:

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = requests.Session()
retries = Retry(total=3, backoff_factor=1, status_forcelist=[502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

def image_converter(url):
    try:
        res = session.get(url, timeout=5)
        if res.status_code == 200:
            print(f"Success: {url}")
            return res.content
    except Exception as e:
        print(f"Failed: {url} - {e}")
    return None

df['image_blob'] = df['image_url'].apply(image_converter)


In [ ]:
df.to_sql('breeds', con=sqlite3.connect('dog_breeds_with_image.db'), if_exists='replace', index=False)

172

In [2]:
con = sqlite3.connect('dog_breeds_with_image.db')
cur = con.cursor()





In [3]:
df_extended = pd.read_csv('akc-data-latest.csv')

df_extended.rename(columns={'Unnamed: 0': 'name'}, inplace=True)

In [4]:
df_extended.to_sql('akc_data', con=con, if_exists='replace', index=False)

277

In [7]:
cur.execute('select * from akc_data')
rows = cur.fetchone()
rows

('Affenpinscher',
 'The Affen’s apish look has been described many ways. They’ve been called “monkey dogs” and “ape terriers.” The French say diablotin moustachu (“mustached little devil”), and Star Wars fans argue whether they look more like Wookies or Ewoks.Standing less than a foot tall, these sturdy terrier-like dogs approach life with great confidence. “This isn’t a breed you train,” a professional dog handler tells us, “He’s like a human. You befriend him.” The dense, harsh coat is described as “neat but shaggy” and comes in several colors; the gait is light and confident. They can be willful and domineering, but mostly Affens are loyal, affectionate, and always entertaining. Affen people say they love being owned by their little monkey dogs. The Affenpinscher: loyal, curious, and famously amusing; this almost-human toy dog is fearless out of all proportion to his size. As with all great comedians, it’s the Affenpinscher’s apparent seriousness of purpose that makes his antics all

In [ ]:
img = Image.open(BytesIO(rows[0]))
img.show()

: 